In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os
import time
import re
from rapidfuzz import fuzz

# Liste des catégories
CATEGORIES = [
    'bio-et-ecologie', 'fruits-et-legumes', 'viandes-et-poissons', 'pains-et-patisseries',
    'cremerie-et-produits-laitiers', 'charcuterie-et-traiteur', 'surgeles', 'epicerie-salee',
    'epicerie-sucree', 'beaute-et-sante', 'boissons', 'nutrition-et-vegetale', 'hygiene-et-beaute',
    'entretien-et-nettoyage', 'animalerie', 'bebe', 'jardin', 'entretien-de-la-maison',
    'maison-et-decoration', 'cuisine', 'gros-electromenager', 'bricolage',
    'velo-trotinettes-et-loisirs', 'smartphones-et-objets-connectes', 'image-et-son',
    'informatique-et-bureau', 'jeux-videos', 'jeux-et-jouets', 'mode-et-bagagerie'
]

# Dossier de sortie
OUTPUT_FOLDER = r"C:/Users/DELL/OneDrive - etu.unistra.fr/Bureau/cours fac/PROGRAMMATION PYTHON/PROJET/Output"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Mode utilisateur
mode = input("Souhaitez-vous scraper des promotions, des produits, ou estimer un prix ? [promotions/products/estimate] : ").strip().lower()
while mode not in ["promotions", "products", "estimate"]:
    mode = input("Veuillez entrer 'promotions', 'products', ou 'estimate' : ").strip().lower()

# Sélection de catégorie si nécessaire
if mode in ["products", "estimate"]:
    print("\nCatégories disponibles :")
    for idx, cat in enumerate(CATEGORIES, 1):
        print(f" {idx}. {cat}")
    while True:
        try:
            cat_choice = int(input("Choisissez le numéro de la catégorie : ").strip())
            if 1 <= cat_choice <= len(CATEGORIES):
                categorie = CATEGORIES[cat_choice - 1]
                break
            else:
                print("Numéro invalide. Réessayez.")
        except ValueError:
            print("Veuillez entrer un numéro valide.")
else:
    categorie = None

# Scraping toutes les pages
def scrap_all_pages(url_base):
    driver = webdriver.Chrome()
    all_data = []
    page = 0
    while True:
        driver.get(f"{url_base}&page={page}")
        time.sleep(5)
        products = driver.find_elements(By.CLASS_NAME, "product-list-grid__item")
        print(f"Scraping page {page + 1} : {len(products)} produits")

        if not products:
            break  # Stop si aucune donnée

        for p in products:
            try:
                name = p.find_element(By.CSS_SELECTOR, "a.product-card-title").text
            except:
                name = None

            try:
                price_block = p.find_element(By.CSS_SELECTOR, "div.product-price__amount--main")
                price_text = price_block.text
                clean_price = re.sub(r'[^\d,\.]', '', price_text)
                price = float(clean_price.replace(',', '.'))
            except:
                price = None

            all_data.append({
                "product_name": name,
                "price": price,
                "page": page + 1
            })

        page += 1  # Passer à la page suivante

    driver.quit()
    return pd.DataFrame(all_data)

# Mode promotions / products
if mode in ["promotions", "products"]:
    if mode == "promotions":
        base_url = "https://www.carrefour.fr/promotions?noRedirect=1"
        df = scrap_all_pages(base_url)
        filename = "carrefour_promotions_allpages.csv"
    else:
        base_url = f"https://www.carrefour.fr/r/{categorie}?noRedirect=1"
        df = scrap_all_pages(base_url)
        filename = f"carrefour_products_{categorie}_allpages.csv"

    # Sauvegarde CSV
    csv_path = os.path.join(OUTPUT_FOLDER, filename)
    df.to_csv(csv_path, index=False)
    print(f"✅ Fichier CSV enregistré : {csv_path}")
    print(df.head())

# Mode estimate
elif mode == "estimate":
    produit_cible = input("Saisissez le nom du produit à estimer : ").strip().lower()
    base_url = f"https://www.carrefour.fr/r/{categorie}?noRedirect=1"
    df = scrap_all_pages(base_url)

    if df.empty:
        print("⚠️ Aucun produit trouvé dans cette catégorie.")
    else:
        df['similarity'] = df['product_name'].apply(lambda x: fuzz.partial_ratio(produit_cible, x.lower()) if pd.notnull(x) else 0)
        similaires = df[df['similarity'] >= 60]

        if similaires.empty:
            print("⚠️ Aucun produit similaire trouvé.")
        else:
            prix_moyen = similaires['price'].mean()
            prix_min = similaires['price'].min()
            prix_max = similaires['price'].max()
            print("\nProduits similaires trouvés :")
            print(similaires[['product_name', 'price']])
            print(f"\nNombre de produits similaires : {len(similaires)}")
            print(f"Prix estimé moyen : {prix_moyen:.2f} €")
            print(f"Prix minimum : {prix_min:.2f} €, Prix maximum : {prix_max:.2f} €")


Souhaitez-vous scraper des promotions, des produits, ou estimer un prix ? [promotions/products/estimate] :  estimate



Catégories disponibles :
 1. bio-et-ecologie
 2. fruits-et-legumes
 3. viandes-et-poissons
 4. pains-et-patisseries
 5. cremerie-et-produits-laitiers
 6. charcuterie-et-traiteur
 7. surgeles
 8. epicerie-salee
 9. epicerie-sucree
 10. beaute-et-sante
 11. boissons
 12. nutrition-et-vegetale
 13. hygiene-et-beaute
 14. entretien-et-nettoyage
 15. animalerie
 16. bebe
 17. jardin
 18. entretien-de-la-maison
 19. maison-et-decoration
 20. cuisine
 21. gros-electromenager
 22. bricolage
 23. velo-trotinettes-et-loisirs
 24. smartphones-et-objets-connectes
 25. image-et-son
 26. informatique-et-bureau
 27. jeux-videos
 28. jeux-et-jouets
 29. mode-et-bagagerie


Choisissez le numéro de la catégorie :  1
Saisissez le nom du produit à estimer :  pomme


Scraping page 1 : 35 produits
Scraping page 2 : 35 produits
Scraping page 3 : 35 produits
Scraping page 4 : 35 produits
Scraping page 5 : 35 produits
Scraping page 6 : 35 produits
Scraping page 7 : 35 produits
Scraping page 8 : 35 produits
Scraping page 9 : 35 produits
Scraping page 10 : 35 produits
Scraping page 11 : 35 produits
Scraping page 12 : 35 produits
Scraping page 13 : 35 produits
Scraping page 14 : 35 produits
Scraping page 15 : 35 produits
Scraping page 16 : 35 produits
Scraping page 17 : 35 produits
Scraping page 18 : 35 produits
Scraping page 19 : 35 produits
Scraping page 20 : 35 produits
Scraping page 21 : 35 produits
Scraping page 22 : 35 produits
Scraping page 23 : 35 produits
Scraping page 24 : 35 produits
Scraping page 25 : 35 produits
Scraping page 26 : 35 produits
Scraping page 27 : 35 produits
Scraping page 28 : 35 produits
Scraping page 29 : 35 produits
Scraping page 30 : 35 produits
Scraping page 31 : 35 produits
Scraping page 32 : 35 produits
Scraping page 33 